## Testing Eleven API

In [3]:
import os
import requests
from dotenv import load_dotenv

def elevenlabs_hello_world():
    # Load environment variables
    load_dotenv()
    elevenlabs_key = os.getenv("ELEVEN_API_KEY")

    if not elevenlabs_key:
        raise RuntimeError("ELEVEN_API_KEY not found in environment")

    # Adam (multilingual) – safe default
    # voice_id = "pNInz6obpgDQGcFmaJgB"
    voice_map = {
                "Spanish": "pNInz6obpgDQGcFmaJgB",  # Adam (multilingual)
                "French": "pNInz6obpgDQGcFmaJgB",
                "German": "pNInz6obpgDQGcFmaJgB",
                "Italian": "pNInz6obpgDQGcFmaJgB",
                "Portuguese": "pNInz6obpgDQGcFmaJgB",
                "Chinese": "pNInz6obpgDQGcFmaJgB",
                "Japanese": "pNInz6obpgDQGcFmaJgB",
                "Korean": "pNInz6obpgDQGcFmaJgB",
                "Arabic": "pNInz6obpgDQGcFmaJgB",
                "Hebrew": "pNInz6obpgDQGcFmaJgB"
            }
    voice_id = voice_map.get("French", "pNInz6obpgDQGcFmaJgB")

    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"

    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": elevenlabs_key,
    }

    payload = {
        "text": "Bonjour, ça va! This is ElevenLabs text to speech.",
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75,
        },
    }

    response = requests.post(url, json=payload, headers=headers, timeout=30)

    if response.status_code != 200:
        raise RuntimeError(
            f"ElevenLabs error {response.status_code}: {response.text}"
        )

    # Save audio
    output_file = "hello_world.mp3"
    with open(output_file, "wb") as f:
        f.write(response.content)

    print(f"✅ Audio saved to {output_file}")

if __name__ == "__main__":
    elevenlabs_hello_world()

✅ Audio saved to hello_world.mp3


In [ ]:
def agent_tts_generator(reading_text: str):
    try:
        key = os.environ.get("ELEVEN_API_KEY")
        if not key:
            return "❌ ELEVEN_API_KEY missing in HF Secrets"
        
        r = requests.post(
            "https://api.elevenlabs.io/v1/text-to-speech/pNInz6obpgDQGcFmaJgB",
            headers={
                "Accept": "audio/mpeg",
                "Content-Type": "application/json",
                "xi-api-key": key,
            },
            json={
                "text": reading_text[:500],
                "model_id": "eleven_multilingual_v2",
            },
            timeout=30,
        )
        
        
        if r.status_code != 200:
            return f"❌ ElevenLabs {r.status_code}: {r.text}"
        
        
        path = "bible_listeing_audio.mp3"
        with open(path, "wb") as f:
            f.write(r.content)
        
        
        return path


    except Exception as e:
        return f"❌ Exception: {e}"



In [9]:
if __name__ == "__main__":
    path=agent_tts_generator(reading_text="hello world! how are you?")
    print(path)

C:\Users\jredo\AppData\Local\Temp\debug.mp3


In [20]:
# import gradio as gr
import openai
import os
from datetime import datetime
import json
from pathlib import Path
import tempfile
import requests

# PDF generation
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from dotenv import load_dotenv


class BibleLanguageLearningSystem:
    """
    Agentic AI system for language learning through Bible study.
    Uses multiple specialized agents to create comprehensive lessons.
    """
    
    def __init__(self, api_key: str, target_language: str = "Spanish", model: str = "gpt-4o-mini", elevenlabs_key: str = None):
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
        self.target_language = target_language
        self.elevenlabs_key = elevenlabs_key
        
    def _call_gpt(self, system_prompt: str, user_message: str, temperature: float = 1.0) -> str:
        """Helper method to call OpenAI API"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                temperature=temperature,
                max_tokens=4000
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error: {str(e)}"
    
    def agent_verse_retriever(self, language_level: str) -> dict:
        """Agent 1: Retrieves verse of the day and meditation paragraph"""
        system_prompt = f"""You are a Bible study coordinator agent. Your role is to:
1. Select an appropriate verse or verses of the day randomly to assure the study is different each time. 
2. Keep the message coherence with the verses.
3. Provide a short meditation paragraph (2-3 sentences) in both English and {self.target_language}
4. Ensure content is appropriate for {language_level} language learners

Return ONLY valid JSON with these exact keys:
- "verse_reference": The Bible verse reference
- "verse_text_english": The verse in English
- "verse_text_{self.target_language.lower()}": The verse in {self.target_language}
- "meditation_english": Meditation in English
- "meditation_{self.target_language.lower()}": Meditation in {self.target_language}"""

        user_message = f"Provide verse of the day with meditation for {self.target_language} at {language_level} level."
        
        response = self._call_gpt(system_prompt, user_message, temperature=0.7)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except:
            return {
                "verse_reference": "John 3:16",
                "verse_text_english": "For God so loved the world...",
                f"verse_text_{self.target_language.lower()}": response[:100],
                "meditation_english": "God's love for humanity.",
                f"meditation_{self.target_language.lower()}": response[100:200] if len(response) > 100 else "Meditación"
            }
    
    def agent_content_creator(self, verse_data: dict, language_level: str) -> dict:
        """Agent 2: Creates reading comprehension paragraph"""
        system_prompt = f"""You are a language learning content creator.
Create a reading comprehension paragraph (150-200 words) in {self.target_language}.

Requirements:
- Appropriate for {language_level} level
- Include theological insights and practical applications
- Use clear, educational language
- Natural pronunciation-friendly text (avoid complex punctuation)

Return ONLY valid JSON with:
- "reading_text_{self.target_language.lower()}": Reading text in {self.target_language}
- "reading_text_english": Reading text in English
- "key_vocabulary": Array of important vocabulary words"""

        user_message = f"""Create content based on:
Verse: {verse_data.get('verse_reference', 'N/A')}
Text: {verse_data.get(f'verse_text_{self.target_language.lower()}', '')}"""

        response = self._call_gpt(system_prompt, user_message, temperature=0.8)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except:
            return {
                f"reading_text_{self.target_language.lower()}": response[:300],
                "reading_text_english": "Reading comprehension text",
                "key_vocabulary": ["faith", "love", "grace"]
            }
    
    def agent_lesson_designer(self, verse_data: dict, reading_data: dict, language_level: str) -> dict:
        """Agent 3: Designs comprehensive lesson"""
        system_prompt = f"""You are an expert language lesson designer for {self.target_language}.
Create a comprehensive lesson for {language_level} level including:

1. READING: 4-5 comprehension questions about the reading text (in {self.target_language})
2. WRITING: 3 writing prompts related to the theme (in {self.target_language})
3. LISTENING: 4 questions about what students should listen for in the audio (in {self.target_language})
4. SPEAKING: 3 speaking prompts for oral practice (in {self.target_language})
5. FILLING: 3-4 fill-in-the-blank sentences using vocabulary from the reading (in {self.target_language})
   - Use ___ to indicate where the word should go
   - Make blanks appropriate for {language_level} level

Return ONLY valid JSON with:
- "reading_exercises": Array of objects with "question" field
- "writing_exercises": Array of objects with "question" field
- "listening_exercises": Array of objects with "question" field
- "speaking_exercises": Array of objects with "question" field
- "filling_exercises": Array of objects with "question" field (sentences with ___ for blanks)

Return ONLY the JSON object, no additional text."""

        user_message = f"""Design lesson based on:
Verse: {verse_data.get('verse_reference')}
Reading: {reading_data.get(f'reading_text_{self.target_language.lower()}', '')[:200]}
Vocabulary: {reading_data.get('key_vocabulary', [])}"""

        response = self._call_gpt(system_prompt, user_message, temperature=0.7)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except:
            return {
                "reading_exercises": [{"question": "¿Cuál es el tema principal del texto?"}],
                "writing_exercises": [{"question": "Escribe sobre tu experiencia personal con este tema."}],
                "listening_exercises": [{"question": "¿Qué palabras clave escuchaste?"}],
                "speaking_exercises": [{"question": "Explica el significado del verso en tus propias palabras."}],
                "filling_exercises": [{"question": "La ___ es importante en la vida cristiana."}]
            }
    
    def agent_answer_key_generator(self, lesson_data: dict, verse_data: dict, reading_data: dict) -> dict:
        """Agent 4: Generates answer key"""
        system_prompt = f"""You are an answer key generator for {self.target_language}.
Provide detailed answers and model responses in {self.target_language}.

For filling exercises, provide ONLY the word(s) that should fill the blank(s).

Return ONLY valid JSON with:
- "reading_exercises": Array with "answer" and "explanation"
- "writing_exercises": Array with "answer" (model response) and "explanation"
- "listening_exercises": Array with "answer" (key points to listen for) and "explanation"
- "speaking_exercises": Array with "answer" (sample response) and "explanation"
- "filling_exercises": Array with "answer" (the missing word/phrase ONLY) and "explanation"
        
Return ONLY the JSON object, no additional text."""

        user_message = f"""Generate answers for:
Exercises: {json.dumps(lesson_data, ensure_ascii=False)[:500]}
Reading context: {reading_data.get(f'reading_text_{self.target_language.lower()}', '')[:300]}
Vocabulary: {reading_data.get('key_vocabulary', [])}"""

        response = self._call_gpt(system_prompt, user_message, temperature=0.5)
        
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            
            return json.loads(json_str)
        except:
            return {
                "reading_exercises": [{"answer": "El tema principal es...", "explanation": "Se encuentra en el párrafo principal"}],
                "writing_exercises": [{"answer": "Ejemplo de respuesta modelo", "explanation": "Respuesta modelo"}],
                "listening_exercises": [{"answer": "Palabras clave: fe, amor, esperanza", "explanation": "Escuchar atentamente"}],
                "speaking_exercises": [{"answer": "El verso significa que...", "explanation": "Guía de conversación"}],
                "filling_exercises": [{"answer": "fe", "explanation": "La palabra correcta es 'fe' según el contexto"}]
            }
    
    def agent_tts_generator(self, reading_text: str, language_level:str) -> str:
        """Agent 5: Generates text-to-speech audio using ElevenLabs"""
        if not self.elevenlabs_key:
            return None
            
        try:
            # Map language to appropriate ElevenLabs voice
            voice_map = {
                "Spanish": "pNInz6obpgDQGcFmaJgB",  # Adam (multilingual)
                "French": "pNInz6obpgDQGcFmaJgB",
                "German": "pNInz6obpgDQGcFmaJgB",
                "Italian": "pNInz6obpgDQGcFmaJgB",
                "Portuguese": "pNInz6obpgDQGcFmaJgB",
                "Chinese": "pNInz6obpgDQGcFmaJgB",
                "Japanese": "pNInz6obpgDQGcFmaJgB",
                "Korean": "pNInz6obpgDQGcFmaJgB",
                "Arabic": "pNInz6obpgDQGcFmaJgB",
                "Hebrew": "pNInz6obpgDQGcFmaJgB"
            }
            
            voice_id = voice_map.get(self.target_language, "pNInz6obpgDQGcFmaJgB")
            
            # ElevenLabs API endpoint
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
            
            headers = {
                "Accept": "audio/mpeg",
                "Content-Type": "application/json",
                "xi-api-key": self.elevenlabs_key
            }
            
            data = {
                "text": reading_text,
                "model_id": "eleven_multilingual_v2",
                "voice_settings": {
                    "stability": 0.5,
                    "similarity_boost": 0.75
                }
            }
            
            response = requests.post(url, json=data, headers=headers)
            
            if response.status_code == 200:
                # Save audio to temporary file
                # temp_dir = tempfile.gettempdir()
                temp_dir = "lessons"
                audio_filename = f"reading_audio_{self.target_language}_{language_level}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp3"
                audio_path = os.path.join(temp_dir, audio_filename)
                # audio_path = audio_filename
                print(f'Audio filepath: {audio_path}')
                
                with open(audio_path, 'wb') as f:
                    f.write(response.content)
                
                return audio_path
            else:
                print(f"ElevenLabs API Error: {response.status_code} - {response.text}")
                return None
                
        except Exception as e:
            print(f"TTS Generation Error: {str(e)}")
            return None
    
    def generate_pdf(self, lesson_content: dict, filename: str = None, language_level:str=None):
        """Generate PDF with all lesson content"""
        if filename is None:
            filename = f"bible_lesson_{self.target_language}_{language_level}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
        
        # Use temporary directory for Hugging Face Spaces
        # temp_dir = tempfile.gettempdir()
        temp_dir = "lessons"
        filepath = os.path.join(temp_dir, filename)
        # filepath = filename
        print(f'Audio filepath: {filepath}')
        
        doc = SimpleDocTemplate(filepath, pagesize=letter,
                                rightMargin=72, leftMargin=72,
                                topMargin=72, bottomMargin=18)
        
        elements = []
        styles = getSampleStyleSheet()
        
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=24,
            textColor='darkblue',
            spaceAfter=30,
            alignment=TA_CENTER
        )
        
        # Title
        title = Paragraph(f"Bible Language Learning Lesson<br/>{self.target_language}", title_style)
        elements.append(title)
        elements.append(Spacer(1, 0.2*inch))
        
        # Date and Level
        date_text = f"Date: {datetime.now().strftime('%B %d, %Y')}<br/>Level: {lesson_content.get('level', 'B1')}"
        elements.append(Paragraph(date_text, styles['BodyText']))
        elements.append(Spacer(1, 0.3*inch))
        
        # Verse
        verse_data = lesson_content.get('verse_data', {})
        elements.append(Paragraph("📖 Verse of the Day", styles['Heading2']))
        elements.append(Spacer(1, 0.1*inch))
        
        verse_ref = verse_data.get('verse_reference', 'N/A')
        elements.append(Paragraph(f"<b>{verse_ref}</b>", styles['BodyText']))
        
        verse_text = verse_data.get(f'verse_text_{self.target_language.lower()}', 'N/A')
        elements.append(Paragraph(f"<i>{verse_text}</i>", styles['BodyText']))
        elements.append(Spacer(1, 0.2*inch))
        
        # Meditation
        meditation = verse_data.get(f'meditation_{self.target_language.lower()}', '')
        if meditation:
            elements.append(Paragraph(f"<b>Meditation:</b> {meditation}", styles['BodyText']))
            elements.append(Spacer(1, 0.3*inch))
        
        # Reading
        reading_data = lesson_content.get('reading_data', {})
        elements.append(Paragraph("📚 Reading Comprehension", styles['Heading2']))
        elements.append(Spacer(1, 0.1*inch))
        
        reading_text = reading_data.get(f'reading_text_{self.target_language.lower()}', 'N/A')
        elements.append(Paragraph(reading_text, styles['BodyText']))
        elements.append(Spacer(1, 0.3*inch))
        
        # Note about audio
        if lesson_content.get('audio_path'):
            elements.append(Paragraph("<b>🔊 Audio available for listening exercise</b>", styles['BodyText']))
            elements.append(Spacer(1, 0.2*inch))
        
        # Vocabulary
        vocab = reading_data.get('key_vocabulary', [])
        if vocab:
            elements.append(Paragraph("<b>Key Vocabulary:</b>", styles['BodyText']))
            vocab_text = ", ".join(vocab) if isinstance(vocab, list) else str(vocab)
            elements.append(Paragraph(vocab_text, styles['BodyText']))
            elements.append(Spacer(1, 0.3*inch))
        
        elements.append(PageBreak())
        
        # Exercises
        lesson_data = lesson_content.get('lesson_data', {})
        self._add_exercises(elements, "📖 Reading Exercises", 
                           lesson_data.get('reading_exercises', []), styles)
        self._add_exercises(elements, "✍️ Writing Exercises", 
                           lesson_data.get('writing_exercises', []), styles)
        self._add_exercises(elements, "👂 Listening Exercises", 
                           lesson_data.get('listening_exercises', []), styles)
        self._add_exercises(elements, "🗣️ Speaking Exercises", 
                           lesson_data.get('speaking_exercises', []), styles)
        self._add_exercises(elements, "✏️ Fill-in-the-Blank Exercises", 
                           lesson_data.get('filling_exercises', []), styles)
        
        elements.append(PageBreak())
        
        # Answers
        elements.append(Paragraph("✅ Answer Key", styles['Heading2']))
        elements.append(Spacer(1, 0.2*inch))
        
        answers = lesson_content.get('answers', {})
        self._add_answers(elements, "Reading Answers", answers.get('reading_exercises', []), styles)
        self._add_answers(elements, "Writing Answers", answers.get('writing_exercises', []), styles)
        self._add_answers(elements, "Listening Answers", answers.get('listening_exercises', []), styles)
        self._add_answers(elements, "Speaking Answers", answers.get('speaking_exercises', []), styles)
        self._add_answers(elements, "Fill-in-the-Blank Answers", answers.get('filling_exercises', []), styles)
        
        doc.build(elements)
        return filepath
    
    def _add_exercises(self, elements, title, exercises, styles):
        """Add exercise section to PDF"""
        elements.append(Paragraph(title, styles['Heading2']))
        elements.append(Spacer(1, 0.1*inch))
        
        if isinstance(exercises, list):
            for i, ex in enumerate(exercises, 1):
                question = ex.get('question', str(ex)) if isinstance(ex, dict) else str(ex)
                elements.append(Paragraph(f"{i}. {question}", styles['BodyText']))
                elements.append(Spacer(1, 0.15*inch))
        
        elements.append(Spacer(1, 0.3*inch))
    
    def _add_answers(self, elements, title, answers, styles):
        """Add answers section to PDF"""
        elements.append(Paragraph(f"<b>{title}</b>", styles['Heading3']))
        elements.append(Spacer(1, 0.1*inch))
        
        if isinstance(answers, list):
            for i, ans in enumerate(answers, 1):
                if isinstance(ans, dict):
                    answer = ans.get('answer', '')
                    explanation = ans.get('explanation', '')
                    text = f"{i}. <b>{answer}</b>"
                    if explanation:
                        text += f" <i>({explanation})</i>"
                else:
                    text = f"{i}. {str(ans)}"
                elements.append(Paragraph(text, styles['BodyText']))
                elements.append(Spacer(1, 0.1*inch))
        
        elements.append(Spacer(1, 0.2*inch))
    
    def run_full_lesson_generation(self, language_level: str = "B1"):
        """Generate complete lesson with progress updates"""
        print(f"0, Starting lesson generation...")
        
        # Step 1
        print(f'0.15 📖 Getting verse of the day...')
        verse_data = self.agent_verse_retriever(language_level)
        
        # Step 2
        print(f"0.30, 📚 Creating reading comprehension...")
        reading_data = self.agent_content_creator(verse_data, language_level)
        
        # Step 3
        print(f"0.45, 🎓 Designing lesson exercises...")
        lesson_data = self.agent_lesson_designer(verse_data, reading_data, language_level)
        
        # Step 4
        print(f"0.60, ✅ Generating answer key...")
        answers = self.agent_answer_key_generator(lesson_data, verse_data, reading_data)
        
        # Step 5: Generate audio
        audio_path = None
        if self.elevenlabs_key:
            print(f"0.75,🔊 Generating audio (this may take a moment)...")
            reading_text = reading_data.get(f'reading_text_{self.target_language.lower()}', '')
            if reading_text:
                audio_path = self.agent_tts_generator(reading_text,language_level)
        
        # Step 6
        print(f"0.90, 📄 Creating PDF...")
        lesson_content = {
            'level': language_level,
            'verse_data': verse_data,
            'reading_data': reading_data,
            'lesson_data': lesson_data,
            'answers': answers,
            'audio_path': audio_path
        }
        
        pdf_path = self.generate_pdf(lesson_content,language_level)
        print(f"1.0, ✨ Complete!")
        
        return lesson_content, pdf_path, audio_path


def format_lesson_display(lesson_content):
    """Format lesson content for display"""
    verse_data = lesson_content.get('verse_data', {})
    reading_data = lesson_content.get('reading_data', {})
    lesson_data = lesson_content.get('lesson_data', {})
    
    # Dynamically get the language key
    lang_keys = [k for k in verse_data.keys() if k.startswith('verse_text_') and k != 'verse_text_english']
    verse_lang_key = lang_keys[0] if lang_keys else 'verse_text_english'
    meditation_lang_key = verse_lang_key.replace('verse_text_', 'meditation_')
    reading_lang_key = verse_lang_key.replace('verse_text_', 'reading_text_')
    
    output = f"""
# 📖 Verse of the Day

**{verse_data.get('verse_reference', 'N/A')}**

*{verse_data.get(verse_lang_key, verse_data.get('verse_text_english', 'N/A'))}*

**Meditation:**
{verse_data.get(meditation_lang_key, verse_data.get('meditation_english', 'N/A'))}

---

# 📚 Reading Comprehension

{reading_data.get(reading_lang_key, reading_data.get('reading_text_english', 'N/A'))}

**Key Vocabulary:** {', '.join(reading_data.get('key_vocabulary', []))}

---

# 📝 Exercises

## 📖 Reading Exercises
"""
    
    for i, ex in enumerate(lesson_data.get('reading_exercises', []), 1):
        output += f"{i}. {ex.get('question', str(ex))}\n"
    
    output += "\n## ✍️ Writing Exercises\n"
    for i, ex in enumerate(lesson_data.get('writing_exercises', []), 1):
        output += f"{i}. {ex.get('question', str(ex))}\n"
    
    output += "\n## 👂 Listening Exercises\n"
    for i, ex in enumerate(lesson_data.get('listening_exercises', []), 1):
        output += f"{i}. {ex.get('question', str(ex))}\n"
    
    output += "\n## 🗣️ Speaking Exercises\n"
    for i, ex in enumerate(lesson_data.get('speaking_exercises', []), 1):
        output += f"{i}. {ex.get('question', str(ex))}\n"

    output += "\n## ✏️ Fill-in-the-Blank Exercises\n"
    for i, ex in enumerate(lesson_data.get('filling_exercises', []), 1):
        output += f"{i}. {ex.get('question', str(ex))}\n"
    
    return output


def generate_lesson(api_key, elevenlabs_key, language, level, model):
    """Main function called by Gradio interface"""
    if not api_key:
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        elevenlabs_key = os.getenv("ELEVEN_API_KEY")
        if not api_key:
            return "⚠️ Please enter your OpenAI API key", None, None
    
    try:
        system = BibleLanguageLearningSystem(
            api_key=api_key,
            target_language=language,
            model=model,
            elevenlabs_key=elevenlabs_key
        )
        
        lesson_content, pdf_path, audio_path = system.run_full_lesson_generation(level)
        
        display_text = format_lesson_display(lesson_content)
        
        return display_text, pdf_path, audio_path
        
    except Exception as e:
        return f"❌ Error: {str(e)}", None, None




In [15]:
api_key = os.getenv("OPENAI_API_KEY")
elevenlabs_key = os.getenv("ELEVEN_API_KEY")
generate_lesson(api_key, elevenlabs_key, "French", "B1", "gpt-4o-mini")

0, Starting lesson generation...
0.15 📖 Getting verse of the day...
0.30, 📚 Creating reading comprehension...
0.45, 🎓 Designing lesson exercises...
0.60, ✅ Generating answer key...
0.75,🔊 Generating audio (this may take a moment)...
Audio filepath: lessons\reading_audio_French_20260129_101854.mp3
0.90, 📄 Creating PDF...
Audio filepath: lessons\bible_lesson_French_20260129_101854.pdf
1.0, ✨ Complete!


('\n# 📖 Verse of the Day\n\n**Philippians 4:13**\n\n*Je peux tout par celui qui me fortifie.*\n\n**Meditation:**\nCe verset nous rappelle que nous pouvons faire face à tous les défis avec la force que Christ nous donne. Quand nous nous sentons faibles ou incertains, nous devons nous rappeler que nous ne sommes pas seuls ; Il est toujours là pour nous aider.\n\n---\n\n# 📚 Reading Comprehension\n\nDans la lettre aux Philippiens, Paul nous enseigne une vérité profonde : "Je peux tout par celui qui me fortifie". Cette phrase nous rappelle que notre force ne vient pas seulement de nous-mêmes, mais de notre foi en Dieu. Quand nous faisons face à des défis dans notre vie, il est important de se souvenir que nous ne sommes pas seuls. La foi nous aide à surmonter les obstacles. Par exemple, si vous devez prendre une décision difficile ou affronter une peur, pensez à prier et à demander du soutien. Cela peut vous donner la force nécessaire pour avancer. De plus, cette citation nous encourage à s

In [16]:
api_key = os.getenv("OPENAI_API_KEY")
elevenlabs_key = os.getenv("ELEVEN_API_KEY")
generate_lesson(api_key, elevenlabs_key, "English", "B1", "gpt-4o-mini")

0, Starting lesson generation...
0.15 📖 Getting verse of the day...
0.30, 📚 Creating reading comprehension...
0.45, 🎓 Designing lesson exercises...
0.60, ✅ Generating answer key...
0.75,🔊 Generating audio (this may take a moment)...
0.90, 📄 Creating PDF...
1.0, ✨ Complete!


("\n# 📖 Verse of the Day\n\n**Philippians 4:13**\n\n*I can do all things through Christ who strengthens me.*\n\n**Meditation:**\nThis verse reminds us that we are never alone. With Christ's strength, we can face any challenge and achieve our goals. Trust in Him, and you will find the power to succeed.\n\n---\n\n# 📚 Reading Comprehension\n\nThe Bible verse Philippians 4:13 says, 'I can do all things through Christ who strengthens me.' This verse teaches us that we can find strength in our faith. It means that no matter what challenges we face, we can overcome them with Christ's help. This idea is important for our daily lives. When we feel weak or unsure, we can remember that we are not alone. We can pray and ask for support. For example, if you have a difficult task at school or work, you can rely on your faith to give you courage and energy. In addition, this verse encourages us to help others. When we see someone struggling, we can remind them of this truth. By sharing our faith, we 

In [6]:
api_key = os.getenv("OPENAI_API_KEY")
elevenlabs_key = os.getenv("ELEVEN_API_KEY")
generate_lesson(api_key, elevenlabs_key, "Catalan", "B1", "gpt-4o-mini")

0, Starting lesson generation...
0.15 📖 Getting verse of the day...
0.30, 📚 Creating reading comprehension...
0.45, 🎓 Designing lesson exercises...
0.60, ✅ Generating answer key...
0.75,🔊 Generating audio (this may take a moment)...
0.90, 📄 Creating PDF...
1.0, ✨ Complete!


('\n# 📖 Verse of the Day\n\n**Philippians 4:13**\n\n*Puc fer totes les coses per mitjà de Crist que em fa fort.*\n\n**Meditation:**\nAquest vers ens recorda que amb fe en Crist, podem superar els reptes. La seva força ens dóna el poder per afrontar situacions difícils amb coratge.\n\n---\n\n# 📚 Reading Comprehension\n\nLa carta als Filipencs, escrita per l\'apòstol Pau, conté un missatge molt poderós. En Filipencs 4:13, Pau diu: "Puc fer totes les coses per mitjà de Crist que em fa fort." Aquesta frase ens recorda que la nostra força no prové només de nosaltres mateixos, sinó de la nostra fe en Jesucrist. Això implica que, en moments de dificultat, podem confiar en la força divina per afrontar els reptes que se\'ns presenten. No importa quines siguin les nostres lluites personals, sabem que no estem sols. Amb la seva ajuda, podem superar obstacles i assolir els nostres objectius. Així, la nostra vida quotidiana pot ser un reflex de la fe i la força que rebem a través de Crist. En la pr

In [9]:
api_key = os.getenv("OPENAI_API_KEY")
elevenlabs_key = os.getenv("ELEVEN_API_KEY")
generate_lesson(api_key, elevenlabs_key, "Spanish", "B1", "gpt-4o-mini")

0, Starting lesson generation...
0.15 📖 Getting verse of the day...
0.30, 📚 Creating reading comprehension...
0.45, 🎓 Designing lesson exercises...
0.60, ✅ Generating answer key...
0.75,🔊 Generating audio (this may take a moment)...
Audio filepath: reading_audio_20260129_101122.mp3
0.90, 📄 Creating PDF...
Audio filepath: bible_lesson_20260129_101122.pdf
1.0, ✨ Complete!


('\n# 📖 Verse of the Day\n\n**Philippians 4:6-7**\n\n*No se inquieten por nada; más bien, en toda ocasión, con oración y ruego, presenten sus peticiones a Dios y denle gracias.*\n\n**Meditation:**\nEste versículo nos recuerda que no debemos preocuparnos. En su lugar, debemos orar y pedir ayuda a Dios. Cuando hacemos esto, podemos sentir paz en nuestros corazones.\n\n---\n\n# 📚 Reading Comprehension\n\nEn Filipenses 4:6-7, se nos invita a no preocuparnos por nada. Esta exhortación nos recuerda la importancia de la oración en nuestra vida diaria. Cuando enfrentamos dificultades, podemos presentar nuestras peticiones a Dios. Esto no solo nos ayuda a liberar nuestras cargas, sino que también fortalece nuestra relación con Él. Al orar, debemos hacerlo con gratitud, reconociendo lo que ya hemos recibido. La paz de Dios, que supera todo entendimiento, nos protegerá. Esto significa que, incluso en momentos de angustia, podemos encontrar tranquilidad. La práctica de la oración y el agradecimien

In [16]:
api_key = os.getenv("OPENAI_API_KEY")
elevenlabs_key = os.getenv("ELEVEN_API_KEY")
generate_lesson(api_key, elevenlabs_key, "English", "C2", "gpt-4o-mini")

0, Starting lesson generation...
0.15 📖 Getting verse of the day...
0.30, 📚 Creating reading comprehension...
0.45, 🎓 Designing lesson exercises...
0.60, ✅ Generating answer key...
0.75,🔊 Generating audio (this may take a moment)...
Audio filepath: lessons\reading_audio_English_20260129_102024.mp3
0.90, 📄 Creating PDF...
Audio filepath: lessons\bible_lesson_English_20260129_102024.pdf
1.0, ✨ Complete!


('\n# 📖 Verse of the Day\n\n**Philippians 4:13**\n\n*I can do all things through Christ who strengthens me.*\n\n**Meditation:**\nThis verse serves as a powerful reminder of the limitless potential we possess when we align ourselves with Christ. It encourages us to embrace challenges, knowing that our strength is not solely our own, but derived from a higher source that empowers us to overcome obstacles.\n\n---\n\n# 📚 Reading Comprehension\n\nThe verse Philippians 4:13, which states, "I can do all things through Christ who strengthens me," encapsulates a profound theological truth about reliance on divine strength. This assertion underscores the centrality of faith in the Christian experience, suggesting that human limitations can be transcended through spiritual empowerment. The Apostle Paul, writing to the Philippians, emphasizes that true capability does not stem from self-sufficiency but from a relationship with Christ. This understanding invites believers to cultivate resilience in